# Data Loading, Cleanup, and Normalization

## Objectives:
1. **Load raw JSON data** into a Pandas DataFrame.
2. **Ensure data integrity**:
   - The `business_id` column should be unique and valid as a primary key.
   - Handle any missing or duplicate values.
3. **Normalize data fields**:
   - Ensure consistency in the `city` column.
   - Split `categories` into separate rows.

## Assumptions:
- The dataset contains `business_id`, `city`, and `categories` columns.
- `categories` is a comma-separated string that needs to be expanded into multiple rows.
- City names may have inconsistencies (e.g., different capitalizations, typos).

## Key Steps:
1. **Load raw_data.json**
2. **Clean business_id** (remove duplicates, handle nulls)
3. **Normalize city names** (formatting, spelling)
4. **Expand categories** (split multi-category rows into single entries)


In [3]:
import pandas as pd

file_path = "../data/raw/raw_data.json"
df = pd.read_json(file_path, lines=True)

df.head()

,address,attributes,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,business_id,hours
0,"1616 Chapala St, Ste 2",{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",Santa Barbara,0,34.426679,-119.711197,"Abby Rappoport, LAC, CMQ",93101,7,5.0,CA,NaN,NaN
1,87 Grasso Plaza Shopping Center,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...",Affton,1,38.551126,-90.335695,The UPS Store,63123,15,3.0,MO,mpf3x-BjTdTEA3yCZrAYPw,"{'Friday': '8:0-18:30', 'Monday': '0:0-0:0', '..."
2,5255 E Broadway Blvd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...",Tucson,0,32.223236,-110.880452,Target,85711,22,3.5,AZ,tUFrWirKiKi_TAnsVWINQQ,"{'Friday': '8:0-23:0', 'Monday': '8:0-22:0', '..."
3,935 Race St,"{'Alcohol': 'u'none'', 'BikeParking': 'True', ...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Philadelphia,1,39.955505,-75.155564,St Honore Pastries,19107,80,4.0,PA,NaN,"{'Friday': '7:0-21:0', 'Monday': '7:0-20:0', '..."
4,101 Walnut St,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Brewpubs, Breweries, Food",Green Lane,1,40.338183,-75.471659,Perkiomen Valley Brewery,18054,13,4.5,PA,mWMc6_wTdE0EUBKIGXDVfA,"{'Friday': '12:0-22:0', 'Saturday': '12:0-22:0..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225572 entries, 0 to 225571
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   address       225572 non-null  object 
 1   attributes    204971 non-null  object 
 2   categories    225415 non-null  object 
 3   city          225572 non-null  object 
 4   is_open       225572 non-null  int64  
 5   latitude      225572 non-null  float64
 6   longitude     225572 non-null  float64
 7   name          225572 non-null  object 
 8   postal_code   225572 non-null  object 
 9   review_count  225572 non-null  int64  
 10  stars         225572 non-null  float64
 11  state         225572 non-null  object 
 12  business_id   158001 non-null  object 
 13  hours         190686 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 24.1+ MB


In [5]:
df.isna().sum()

address             0
attributes      20601
categories        157
city                0
is_open             0
latitude            0
longitude           0
name                0
postal_code         0
review_count        0
stars               0
state               0
business_id     67571
hours           34886
dtype: int64

In [7]:
import json

# Convert columns containing dictionaries into JSON strings for better storage and processing.
columns_to_convert = ["attributes", "hours"]

for col in columns_to_convert:
    df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)


In [8]:
df.drop_duplicates(inplace=True)
print(f"✅ After removing duplicates, {len(df)} rows remain")

✅ After removing duplicates, 178399 rows remain


In [9]:
df_sorted = df.sort_values(by="name")
df_sorted

,address,attributes,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,business_id,hours
69775,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}","Preschools, Education, Local Services, Child C...",Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,NaN,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
33991,6802 Ridge Ave,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Beauty & Spas, Barbers",Philadelphia,1,40.042395,-75.226518,Joe's Throwback Barber Shop,19128,17,5.0,PA,t7vVRlyF9_lxuL1auqVchg,"{""Friday"": ""7:0-19:0"", ""Monday"": ""0:0-0:0"", ""S..."
144788,112 S Center St,"{""BusinessAcceptsCreditCards"": ""True"", ""Busine...","Barbers, Beauty & Spas",Plainfield,1,39.702890,-86.402923,Leland's Barbershop,46168,6,4.5,IN,NaN,"{""Friday"": ""7:0-17:0"", ""Saturday"": ""7:0-14:0"",..."
57020,8131 Kingston St,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Arts & Entertainment, Venues & Event Spaces, E...",Avon,0,39.759701,-86.378301,Xtreme Laser Tag Avon,46123,12,4.0,IN,f9pqgFRKMoN75uqzxd7KHw,"{""Friday"": ""17:0-23:0"", ""Saturday"": ""10:0-23:0..."
33505,14351 N Dale Mabry Hwy,"{""Ambience"": ""{'touristy': False, 'hipster': F...","Pizza, Breakfast & Brunch, Restaurants, Food, ...",Tampa,0,28.077936,-82.506174,"""Genuino"" Italian Cafe'",33618,60,4.5,FL,ihYl-uK0PYNPeFskcNAr-g,"{""Friday"": ""9:0-23:0"", ""Saturday"": ""9:0-23:0"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31365,900 Terry Pkwy,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Bakeries, Food, Desserts, Patisserie/Cake Shop",Terrytown,1,29.892304,-90.031530,éclair délicieux,70056,40,4.5,LA,2KlvBObysXe00p7phB45_A,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
85930,109 S 10th St,"{""Alcohol"": ""u'beer_and_wine'"", ""Ambience"": ""{...","Restaurants, Cafes, Food, Coffee & Tea",Boise,1,43.616756,-116.206361,ā café,83702,48,5.0,ID,VUKhLVz4X4zVS5wwjAxQjg,"{""Friday"": ""8:0-15:0"", ""Monday"": ""0:0-0:0"", ""S..."
17887,"943 E University Blvd, Ste 101",NaN,"Apartments, University Housing, Home Services,...",Tucson,1,32.232275,-110.957672,ōLiv Tucson,85719,11,1.0,AZ,NaN,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
129908,750 Ladera Ln,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Real Estate, Home Services, Churches, Fitness ...",Montecito,1,34.440557,-119.578843,​Silent Stay Meditation & Retreat Center,93108,33,5.0,CA,QB6O8M_lW3DSvXHhbylEjQ,"{""Friday"": ""9:0-17:0"", ""Monday"": ""9:0-17:0"", ""..."


In [10]:
df_sorted.isna().sum()

address             0
attributes      16262
categories        123
city                0
is_open             0
latitude            0
longitude           0
name                0
postal_code         0
review_count        0
stars               0
state               0
business_id     59621
hours           27498
dtype: int64

In [11]:
duplicates_missing_id = df_sorted[df_sorted.duplicated(subset=df_sorted.columns.difference(["business_id"]), keep=False)]
print(f"Found {len(duplicates_missing_id)} rows where everything is identical except for business_id.")


Found 56106 rows where everything is identical except for business_id.


In [13]:
# Grouping to keep only unique combinations while retaining business_id
df_filled_ids = df_sorted.dropna(subset=["business_id"]).drop_duplicates(subset=df_sorted.columns.difference(["business_id"]).tolist())



In [14]:
df_sorted = df_sorted.merge(df_filled_ids[["business_id"] + df_sorted.columns.difference(["business_id"]).tolist()],
                            on=df_sorted.columns.difference(["business_id"]).tolist(),
                            how="left",
                            suffixes=("", "_filled"))

# If business_id is missing, fill it with the matched one
df_sorted["business_id"] = df_sorted["business_id"].fillna(df_sorted["business_id_filled"])

# Remove the temporary column
df_sorted.drop(columns=["business_id_filled"], inplace=True)


In [15]:
df_sorted.isna().sum()

address             0
attributes      16262
categories        123
city                0
is_open             0
latitude            0
longitude           0
name                0
postal_code         0
review_count        0
stars               0
state               0
business_id     31568
hours           27498
dtype: int64

In [16]:
df_sorted.drop_duplicates(inplace=True)
df_sorted.isna().sum()

address             0
attributes      13744
categories        103
city                0
is_open             0
latitude            0
longitude           0
name                0
postal_code         0
review_count        0
stars               0
state               0
business_id     31568
hours           23223
dtype: int64

In [17]:
# Remove rows with missing business_id and duplicates based on key fields
df_filled_ids = df_sorted.dropna(subset=["business_id"]).drop_duplicates(subset=["name", "city", "address"])

# Merge with the cleaned dataset to fill missing business_id values
df_sorted = df_sorted.merge(df_filled_ids[["name", "city", "address", "business_id"]],
                            on=["name", "city", "address"],
                            how="left",
                            suffixes=("", "_filled"))

# Fill missing business_id values from matched records
df_sorted["business_id"] = df_sorted["business_id"].fillna(df_sorted["business_id_filled"])
df_sorted.drop(columns=["business_id_filled"], inplace=True)

# Check if any NaN values remain in business_id
print(f" Remaining {df_sorted['business_id'].isna().sum()} records with NaN in business_id after update.")


 Remaining 31534 records with NaN in business_id after update.


In [18]:
df_sorted["business_id"] = df_sorted["business_id"].fillna("Unknown_" + df_sorted.index.to_series().astype(str))

## Ensuring Uniqueness and Completeness of `business_id`

### 🔹 Goal:
The goal of this transformation is to ensure that `business_id` is **unique and complete**, even if some records are missing their original `business_id`. The approach follows three key steps:


### 1️⃣ Identify and Clean Valid `business_id` Records
- **Remove rows where `business_id` is missing (`NaN`)** to ensure only valid identifiers are used.
- **Eliminate duplicate records** based on key fields: `"name"`, `"city"`, `"address"`.
- This step ensures that we retain a set of **unique, valid business identifiers**.


### 2️⃣ Merge Back Missing `business_id` Values Using Matching Fields
- Some businesses may exist in the dataset but have **missing `business_id`**.
- We attempt to **fill missing `business_id` values** by matching on `("name", "city", "address")`.
- If a record **without `business_id`** has an **exact match** in the cleaned dataset, we inherit that `business_id`.


### 3️⃣ Handle Remaining Missing `business_id` Values
- If a record **still** has a missing `business_id` after the merge, we generate a **temporary identifier**:

  "Unknown_" + index number


In [19]:
df_sorted.isna().sum()

address             0
attributes      13744
categories        103
city                0
is_open             0
latitude            0
longitude           0
name                0
postal_code         0
review_count        0
stars               0
state               0
business_id         0
hours           23223
dtype: int64

In [20]:
df_sorted

,address,attributes,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,business_id,hours
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}","Preschools, Education, Local Services, Child C...",Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
1,6802 Ridge Ave,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Beauty & Spas, Barbers",Philadelphia,1,40.042395,-75.226518,Joe's Throwback Barber Shop,19128,17,5.0,PA,t7vVRlyF9_lxuL1auqVchg,"{""Friday"": ""7:0-19:0"", ""Monday"": ""0:0-0:0"", ""S..."
2,112 S Center St,"{""BusinessAcceptsCreditCards"": ""True"", ""Busine...","Barbers, Beauty & Spas",Plainfield,1,39.702890,-86.402923,Leland's Barbershop,46168,6,4.5,IN,Unknown_2,"{""Friday"": ""7:0-17:0"", ""Saturday"": ""7:0-14:0"",..."
3,8131 Kingston St,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Arts & Entertainment, Venues & Event Spaces, E...",Avon,0,39.759701,-86.378301,Xtreme Laser Tag Avon,46123,12,4.0,IN,f9pqgFRKMoN75uqzxd7KHw,"{""Friday"": ""17:0-23:0"", ""Saturday"": ""10:0-23:0..."
4,14351 N Dale Mabry Hwy,"{""Ambience"": ""{'touristy': False, 'hipster': F...","Pizza, Breakfast & Brunch, Restaurants, Food, ...",Tampa,0,28.077936,-82.506174,"""Genuino"" Italian Cafe'",33618,60,4.5,FL,ihYl-uK0PYNPeFskcNAr-g,"{""Friday"": ""9:0-23:0"", ""Saturday"": ""9:0-23:0"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,900 Terry Pkwy,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Bakeries, Food, Desserts, Patisserie/Cake Shop",Terrytown,1,29.892304,-90.031530,éclair délicieux,70056,40,4.5,LA,2KlvBObysXe00p7phB45_A,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
150342,109 S 10th St,"{""Alcohol"": ""u'beer_and_wine'"", ""Ambience"": ""{...","Restaurants, Cafes, Food, Coffee & Tea",Boise,1,43.616756,-116.206361,ā café,83702,48,5.0,ID,VUKhLVz4X4zVS5wwjAxQjg,"{""Friday"": ""8:0-15:0"", ""Monday"": ""0:0-0:0"", ""S..."
150343,"943 E University Blvd, Ste 101",NaN,"Apartments, University Housing, Home Services,...",Tucson,1,32.232275,-110.957672,ōLiv Tucson,85719,11,1.0,AZ,Unknown_150343,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
150344,750 Ladera Ln,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Real Estate, Home Services, Churches, Fitness ...",Montecito,1,34.440557,-119.578843,​Silent Stay Meditation & Retreat Center,93108,33,5.0,CA,QB6O8M_lW3DSvXHhbylEjQ,"{""Friday"": ""9:0-17:0"", ""Monday"": ""9:0-17:0"", ""..."


In [22]:
import re
from thefuzz import process


# Clean city names
def clean_city_name(city):
    """
    Cleans city names by:
    - Removing leading/trailing spaces
    - Replacing multiple spaces with a single space
    - Converting to Title Case
    - Removing dots and commas
    """
    if pd.isna(city):
        return None
    city = city.strip()
    city = re.sub(r"\s+", " ", city)
    city = city.title()
    city = re.sub(r"[.,]", "", city)
    return city

df_sorted["city"] = df_sorted["city"].apply(clean_city_name)

# Standardize city names using fuzzy matching
unique_cities = df_sorted["city"].dropna().unique()
city_mapping = {}

for city in unique_cities:
    result = process.extractOne(city, city_mapping.keys(), score_cutoff=80)

    if result:
        best_match, score = result
        city_mapping[city] = best_match  # Assign the most appropriate name
    else:
        city_mapping[city] = city   #Keep original name if no match found

df_sorted["city"] = df_sorted["city"].map(city_mapping)

print(f"Total unique cities after processing: {df_sorted['city'].nunique()}")


Total unique cities after processing: 755


### 1️⃣ **Cleaning City Names**
- Remove **leading and trailing spaces**.
- Replace **multiple spaces** with a **single space**.
- Convert names to **Title Case**.
- Remove **dots and commas** for consistency.

### 2️⃣ **Standardizing City Names with Fuzzy Matching**
To handle **spelling variations and similar city names**, we use **fuzzy matching**:
- Extract a list of **unique city names**.
- Compare each city against the existing standardized names.
- If a match **above 80% similarity** is found, use the best match.
- Otherwise, retain the original city name.


In [23]:
for city in sorted(df_sorted["city"].unique()):
    print(city)

Abington
Afton
Alberta Park Industrial
Aldan
Algiers
Aliso Viejo
Alloway
Almonesson
Alton
Ambler
Andalusia
Antioch
Apollo Beach
Apopka
Arabi
Ardmore
Arizona
Arnold
Ashland
Ashland City
Aston
Atco
Audubon
Austin
Avon
Bala Cynwyd
Ballwin
Balm
Bargersville
Barnhart
Barto
Bayonet Point
Bear
Beaumont
Beech Grove
Bel Ridge
Belle Chasse
Belle Meade
Belleair
Belleair Bluffs
Bellefontaine
Bellefonte
Belleville
Bellevue
Bellmawr
Bellville
Belmont Hills
Bennington
Bensalem
Berkeley
Berlin
Berwyn
Bethel Township
Beverly
Birchrunville
Black Jack
Blackwood
Blooming Glen
Blue Bell
Blvd
Boise
Boone
Boothwyn
Bordentown
Boston
Boulevard
Boutte
Boyertown
Bradenton
Brandon
Breckenridge Hills
Brentwood
Brentwood - Cool Springs
Bridge City
Bridgeport
Bridgeton
Bristol
Brookhaven
Brooklawn
Brooksville
Broomall
Brownsburg
Bryn Athyn
Bryn Mawr
Buckingham
Bucks
Burlington
Bywater
Cahokia
Caln
Camby
Camden
Cane Ridge
Carmel
Carneys Point
Carpinteria
Carrollwood
Casas Adobes
Caseyville
Catalina
Cedar Brook
Cedars

In [24]:
df_sorted

,address,attributes,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,business_id,hours
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}","Preschools, Education, Local Services, Child C...",Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
1,6802 Ridge Ave,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Beauty & Spas, Barbers",Philadelphia,1,40.042395,-75.226518,Joe's Throwback Barber Shop,19128,17,5.0,PA,t7vVRlyF9_lxuL1auqVchg,"{""Friday"": ""7:0-19:0"", ""Monday"": ""0:0-0:0"", ""S..."
2,112 S Center St,"{""BusinessAcceptsCreditCards"": ""True"", ""Busine...","Barbers, Beauty & Spas",Plainfield,1,39.702890,-86.402923,Leland's Barbershop,46168,6,4.5,IN,Unknown_2,"{""Friday"": ""7:0-17:0"", ""Saturday"": ""7:0-14:0"",..."
3,8131 Kingston St,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Arts & Entertainment, Venues & Event Spaces, E...",Avon,0,39.759701,-86.378301,Xtreme Laser Tag Avon,46123,12,4.0,IN,f9pqgFRKMoN75uqzxd7KHw,"{""Friday"": ""17:0-23:0"", ""Saturday"": ""10:0-23:0..."
4,14351 N Dale Mabry Hwy,"{""Ambience"": ""{'touristy': False, 'hipster': F...","Pizza, Breakfast & Brunch, Restaurants, Food, ...",Tampa,0,28.077936,-82.506174,"""Genuino"" Italian Cafe'",33618,60,4.5,FL,ihYl-uK0PYNPeFskcNAr-g,"{""Friday"": ""9:0-23:0"", ""Saturday"": ""9:0-23:0"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,900 Terry Pkwy,"{""BikeParking"": ""True"", ""BusinessAcceptsBitcoi...","Bakeries, Food, Desserts, Patisserie/Cake Shop",Terrytown,1,29.892304,-90.031530,éclair délicieux,70056,40,4.5,LA,2KlvBObysXe00p7phB45_A,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
150342,109 S 10th St,"{""Alcohol"": ""u'beer_and_wine'"", ""Ambience"": ""{...","Restaurants, Cafes, Food, Coffee & Tea",Boise,1,43.616756,-116.206361,ā café,83702,48,5.0,ID,VUKhLVz4X4zVS5wwjAxQjg,"{""Friday"": ""8:0-15:0"", ""Monday"": ""0:0-0:0"", ""S..."
150343,"943 E University Blvd, Ste 101",NaN,"Apartments, University Housing, Home Services,...",Tucson,1,32.232275,-110.957672,ōLiv Tucson,85719,11,1.0,AZ,Unknown_150343,"{""Friday"": ""10:0-19:0"", ""Monday"": ""0:0-0:0"", ""..."
150344,750 Ladera Ln,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Real Estate, Home Services, Churches, Fitness ...",Montecito,1,34.440557,-119.578843,​Silent Stay Meditation & Retreat Center,93108,33,5.0,CA,QB6O8M_lW3DSvXHhbylEjQ,"{""Friday"": ""9:0-17:0"", ""Monday"": ""9:0-17:0"", ""..."


In [25]:
# Split categories by comma and expand them into separate rows
df_exploded = df_sorted.assign(categories=df_sorted["categories"].str.split(",")).explode("categories")

# Remove extra spaces and convert to Title Case
df_exploded["categories"] = df_exploded["categories"].str.strip().str.title()


print(f"Total unique categories after processing: {df_exploded['categories'].nunique()}")

df_sorted = df_exploded

Total unique categories after processing: 1311


### 1️⃣ **Splitting Categories into Separate Rows**
- The original dataset may have multiple categories **stored in a single row**, separated by commas.
- We use `.str.split(",")` to break **multi-category values** into **lists**.
- The `.explode("categories")` function expands these **lists into separate rows**.


### 2️⃣ **Cleaning Category Names**
After expansion, we further clean category names by:
- **Removing leading/trailing spaces** to avoid inconsistencies.
- **Converting names to Title Case**.

In [26]:
df_sorted

,address,attributes,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars,state,business_id,hours
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}",Preschools,Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}",Education,Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}",Local Services,Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
0,830 Kirkwood Ave,"{""BusinessAcceptsCreditCards"": ""True""}",Child Care & Day Care,Nashville,1,36.121090,-86.779848,Grow Academy,37204,6,3.5,TN,Unknown_0,"{""Friday"": ""7:0-18:0"", ""Monday"": ""7:0-18:0"", ""..."
1,6802 Ridge Ave,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...",Beauty & Spas,Philadelphia,1,40.042395,-75.226518,Joe's Throwback Barber Shop,19128,17,5.0,PA,t7vVRlyF9_lxuL1auqVchg,"{""Friday"": ""7:0-19:0"", ""Monday"": ""0:0-0:0"", ""S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150345,1010 Anacapa St,"{""AcceptsInsurance"": ""False"", ""BusinessAccepts...",Massage,Santa Barbara,0,34.422961,-119.700986,​​Transformational Abdominal Massage by Jada D...,93101,11,5.0,CA,Unknown_150345,NaN
150345,1010 Anacapa St,"{""AcceptsInsurance"": ""False"", ""BusinessAccepts...",Naturopathic/Holistic,Santa Barbara,0,34.422961,-119.700986,​​Transformational Abdominal Massage by Jada D...,93101,11,5.0,CA,Unknown_150345,NaN
150345,1010 Anacapa St,"{""AcceptsInsurance"": ""False"", ""BusinessAccepts...",Health & Medical,Santa Barbara,0,34.422961,-119.700986,​​Transformational Abdominal Massage by Jada D...,93101,11,5.0,CA,Unknown_150345,NaN
150345,1010 Anacapa St,"{""AcceptsInsurance"": ""False"", ""BusinessAccepts...",Traditional Chinese Medicine,Santa Barbara,0,34.422961,-119.700986,​​Transformational Abdominal Massage by Jada D...,93101,11,5.0,CA,Unknown_150345,NaN


# ATC 1 Data Preparation and Cleanup

## Objectives:
- Deduplicate ATC 1 definitions table while preserving the most accurate and up-to-date information.
- Ensure all transformations follow best practices.
- Prepare the dataset for a **JOIN operation** with a large hospital dataset (~500k rows).
- Optimize processing for **datalake** scenarios.

## Assumptions:
- The dataset contains `valid_from` and `valid_to` columns for time-based filtering.
- Some ATC 1 definitions might have multiple entries due to updates.
- The hospital dataset is large, so scalability is a concern.

## Key Steps:
1. **Load raw data**.
2. **Convert date columns to DateType** for proper time handling.
3. **Deduplicate records** while keeping the most recent versions.
4. **Perform a JOIN** with the hospital dataset.
5. **Optimize for scalability**, ensuring the approach works for big data processing.


In [1]:
import pandas as pd
from datetime import datetime


In [4]:

atc = [
    ("1900-01-01", "9999-12-31", "A", "Maagdarmkanaal en Metabolisme", "Alimentary Tract and Metabolism"),
    ("1900-01-01", "9999-12-31", "B", "Bloed en Bloedvormende Organen", "Blood and Blood Forming Organs"),
    ("1900-01-01", "9999-12-31", "C", "Hartvaatstelsel", "Cardiovascular System"),
    ("1900-01-01", "9999-12-31", "D", "Dermatologica", "Dermatologicals"),
    ("1900-01-01", "9999-12-31", "G", "Urogenitale Stelsel en Geslachtshormonen", "Genito Urinary System and Sex Hormones"),
    ("1900-01-01", "2022-01-30", "H", "Systemische Hormoonpreparaten, Excl Geslachtshormonen", "Systemic Hormonal Prep,Excl Sex Hormones"),
    ("2022-01-31", "9999-12-31", "H", "Systemische Hormoonpreparaten, Excl Geslachtshormonen", "Systemic Hormonal Preparations, Excl. Sex Hormones and Insulins"),
    ("1900-01-01", "2022-01-30", "J", "Antimicrobiele Middelen Voor Systemisch Gebruik", "General Antiinfectives for Systemic Use"),
    ("2022-01-31", "9999-12-31", "J", "Antimicrobiele Middelen Voor Systemisch Gebruik", "Antiinfectives for Systemic Use"),
    ("1900-01-01", "9999-12-31", "L", "Oncolytica en Immunomodulantia", "Antineoplastic and Immunomodulating Agents"),
    ("1900-01-01", "9999-12-31", "M", "Skeletspierstelsel", "Musculo-Skeletal System"),
    ("1900-01-01", "9999-12-31", "N", "Zenuwstelsel", "Nervous System"),
    ("1900-01-01", "2022-01-30", "P", "Antiparasitica, Insecticiden en Insectenwerende Middelen", "Antiparasitic Products,Insecticides and Repellents"),
    ("2022-01-31", "9999-12-31", "P", "Antiparasitica, Insecticiden en Insectenwerende Middelen", "Antiparasitic Products, Insecticides and Repellents"),
    ("1900-01-01", "9999-12-31", "R", "Ademhalingsstelsel", "Respiratory System"),
    ("1900-01-01", "9999-12-31", "S", "Zintuiglijke Organen", "Sensory Organs"),
    ("1900-01-01", "9999-12-31", "V", "Diverse Middelen", "Various"),
    ("1900-01-01", "2024-12-30", "Y", "Niet Ingevuld", "Not filled in"),
    ("2024-12-31", "9999-12-31", "Y", "Niet Ingevuld", None),
    ("1900-01-01", "2024-12-30", "Z", "Niet Van Toepassing", "Not applicable"),
    ("2024-12-31", "9999-12-31", "Z", "Niet Van Toepassing", None)
]

df = pd.DataFrame(atc, columns=["valid_from", "valid_to", "code", "description", "description_en"])

# Convert date columns from string to datetime format
df["valid_to"] = df["valid_to"].replace("9999-12-31", "2099-12-31")

# Převod sloupců na datetime a nastavení přesnosti na den
df["valid_from"] = pd.to_datetime(df["valid_from"]).astype("datetime64[ms]")
df["valid_to"] = pd.to_datetime(df["valid_to"]).astype("datetime64[ms]")

# Nastavení NaT ve sloupci "valid_to", pokud hodnota odpovídá '9999-12-31'
df.loc[df["valid_to"] == pd.Timestamp("9999-12-31"), "valid_to"] = pd.NaT
df.head()


,valid_from,valid_to,code,description,description_en
0,1900-01-01,2099-12-31,A,Maagdarmkanaal en Metabolisme,Alimentary Tract and Metabolism
1,1900-01-01,2099-12-31,B,Bloed en Bloedvormende Organen,Blood and Blood Forming Organs
2,1900-01-01,2099-12-31,C,Hartvaatstelsel,Cardiovascular System
3,1900-01-01,2099-12-31,D,Dermatologica,Dermatologicals
4,1900-01-01,2099-12-31,G,Urogenitale Stelsel en Geslachtshormonen,Genito Urinary System and Sex Hormones


In [5]:
# Deduplicate by keeping the most recent valid_from record for each code
df = df.sort_values(by=["code", "valid_from"], ascending=[True, False])
df = df.drop_duplicates(subset=["code"], keep="first")

# Display results
df


,valid_from,valid_to,code,description,description_en
0,1900-01-01,2099-12-31,A,Maagdarmkanaal en Metabolisme,Alimentary Tract and Metabolism
1,1900-01-01,2099-12-31,B,Bloed en Bloedvormende Organen,Blood and Blood Forming Organs
2,1900-01-01,2099-12-31,C,Hartvaatstelsel,Cardiovascular System
3,1900-01-01,2099-12-31,D,Dermatologica,Dermatologicals
4,1900-01-01,2099-12-31,G,Urogenitale Stelsel en Geslachtshormonen,Genito Urinary System and Sex Hormones
6,2022-01-31,2099-12-31,H,"Systemische Hormoonpreparaten, Excl Geslachtsh...","Systemic Hormonal Preparations, Excl. Sex Horm..."
8,2022-01-31,2099-12-31,J,Antimicrobiele Middelen Voor Systemisch Gebruik,Antiinfectives for Systemic Use
9,1900-01-01,2099-12-31,L,Oncolytica en Immunomodulantia,Antineoplastic and Immunomodulating Agents
10,1900-01-01,2099-12-31,M,Skeletspierstelsel,Musculo-Skeletal System
11,1900-01-01,2099-12-31,N,Zenuwstelsel,Nervous System


In [6]:
# Simulated hospital dataset ( it would contain ~500k rows)
hospital_data = [
    ("A", "Some medical data 1"),
    ("B", "Some medical data 2"),
    ("C", "Some medical data 3"),
    ("H", "Some medical data 4"),
]

hospital_df = pd.DataFrame(hospital_data, columns=["code", "hospital_info"])

# Perform a left JOIN to retain all ATC 1 data
merged_df = df.merge(hospital_df, on="code", how="left")

# Display first rows
merged_df.head()


,valid_from,valid_to,code,description,description_en,hospital_info
0,1900-01-01,2099-12-31,A,Maagdarmkanaal en Metabolisme,Alimentary Tract and Metabolism,Some medical data 1
1,1900-01-01,2099-12-31,B,Bloed en Bloedvormende Organen,Blood and Blood Forming Organs,Some medical data 2
2,1900-01-01,2099-12-31,C,Hartvaatstelsel,Cardiovascular System,Some medical data 3
3,1900-01-01,2099-12-31,D,Dermatologica,Dermatologicals,NaN
4,1900-01-01,2099-12-31,G,Urogenitale Stelsel en Geslachtshormonen,Genito Urinary System and Sex Hormones,NaN


In [ ]:
# Save as Parquet for efficient storage
merged_df.to_parquet("processed_atc_data.parquet", engine="pyarrow", index=False)
print("✅ Data successfully saved to Parquet!")


## Summary of Data Processing Decisions

✔ **Converted date columns to `datetime` format** for proper time-based analysis.

✔ **Removed duplicates** while keeping the latest valid records.

✔ **Used efficient JOIN** (left merge) to combine with hospital data.

✔ **Saved in Parquet format** to ensure scalability for large datasets.



# 📌 Data Lake Use Case: Scalable and Efficient Data Ingestion & Serving

## 1️⃣ Data Ingestion (Efficient and Scalable Data Loading)

### 🔹 Choosing the Right Storage Format

| File Format | Advantages | Use Case |
|------------|------------|------------|
| **Parquet** | Columnar format, high compression, fast read/write | Best for analytical queries, data lakes |
| **CSV / JSON** | Human-readable, flexible, but inefficient | Suitable for raw data staging only |

📌 **Best Choice:** **Parquet** due to its **compression and analytical efficiency**.

### 🔹 Batch vs. Streaming Ingestion
Depending on the data source and business needs, we can use **Batch Processing** or **Streaming Processing**.

| **Ingestion Type** | **Description** | **Tools & Technologies** | **Example Use Case** |
|------------------|----------------------|-------------------|------------------|
| **Batch Processing** | Loads large datasets at scheduled intervals | Apache Airflow, AWS Glue, Azure Data Factory, Spark | Daily data ingestion from CRM, ERP, and databases |
| **Streaming Processing** | Ingests data in real-time as events occur | Apache Kafka, Spark Streaming, Apache Flink, AWS Kinesis | IoT sensors, user clickstreams, fraud detection |

### 🔹 Implementing Batch Processing
📌 **Workflow:**

1️⃣ Extract data from **databases, APIs, or logs**.

2️⃣ Apply initial **data validation and cleaning**.

3️⃣ Convert into **Parquet format** and store in the **Raw Data Zone**.

4️⃣ Schedule **daily processing** with **Apache Airflow or AWS Glue**.

### 🔹 Implementing Streaming Processing
📌 **Workflow:**

1️⃣ Data is ingested **in real-time** via **Kafka, AWS Kinesis, or Event Hubs**.

2️⃣ **Spark Streaming** processes data and applies transformations.

3️⃣ Data is stored in **Delta Lake** for real-time access.

## 2️⃣ Data Serving (Providing Access to Processed Data)

### 🔹 Key Access Methods
| Serving Method | Description | Use Case |
|---------------|------------|------------|
| **BI Dashboards** | Connects to Tableau, Power BI, Looker | Real-time business intelligence |
| **REST API Access** | Allows external applications to retrieve data | Web applications, automation |
| **SQL Querying** | Enables ad-hoc queries via Presto, Athena, DuckDB | Data exploration, analytics |
| **Automated Reports** | Scheduled reports to business teams | Monthly revenue, user behavior reports |

### 🔹 Optimizing Query Performance
To ensure **fast queries**, we apply:

✔ **Indexing** → Faster lookups (Z-Order, Bloom Filters).

✔ **Partitioning** → Efficient query pruning (`partition by year, month`).

✔ **Caching** → Store frequently accessed data in memory (`Spark Cache`).


# 📌 Enhanced Data Lake Architecture

## 1️⃣ Data Sources
```
├── Transactional Systems (OLTP, ODS) – Databases, CRM, ERP
├── Enterprise Data Warehouse (EDW) – Centralized Data Storage
├── Structured Data – SQL, CSV, JSON
├── Unstructured Data – Logs, Files, API Data
├── Streaming Data – Kafka, IoT, Clickstream
↓
```

## 2️⃣ Data Ingestion (Batch & Streaming)
```
├── Batch Processing – Apache Airflow, AWS Glue, Azure Data Factory
├── Streaming Processing – Kafka, Flink, Spark Streaming
├── Initial Validation & Filtering of Incoming Data
↓
```

### 📍 Raw Data Zone
```
├── Stored in Parquet
↓
```

## 3️⃣ Data Processing (Cleaning & Enrichment)
```
├── Data Wrangling (Spark, Pandas)
├── Data Quality Management (Apache Deequ)
├── Data Transformation (Delta Lake, dbt)
├── Tokenization & Encryption – Securing Sensitive Data
↓
```

### 📍 Refined Data Zone
```
├── Stored in Delta Lake / Iceberg
↓
```

## 4️⃣ Data Storage (Optimized Storage)
```
├── Metadata Management – Data Catalog (Glue, Apache Atlas)
├── Partitioning & Indexing – Z-Order, Bloom Filters for Performance Optimization
↓
```

## 5️⃣ Data Serving (Consumption & Analytics)
```
├── BI Dashboards – Tableau, Power BI, Looker
├── Data API & Access – GraphQL, REST, Athena, Presto
├── Machine Learning & AI – Used by Data Scientists for Model Training
├── Automated Reports – Business Report Generation